In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from sklearn.impute import SimpleImputer

In [3]:
historical_weather = pd.read_csv(r"C:\0A___________________________________\Predictor 1.0\historical_weather.csv")
test_data = pd.read_csv(r"C:\0A___________________________________\Predictor 1.0\submission_key.csv")

In [4]:
historical_weather.drop(columns=['snow_depth_mm'], inplace=True)

In [5]:
historical_weather['date'] = pd.to_datetime(historical_weather['date'])
test_data['date'] = pd.to_datetime(test_data['date'])

In [6]:
historical_weather['city_id'] = historical_weather['city_id'].str.extract('(\d+)').astype(int)
test_data['city_id'] = test_data['city_id'].str.extract('(\d+)').astype(int)

In [7]:
imputer = SimpleImputer(strategy='mean')

In [8]:
historical_weather[['avg_temp_c', 'min_temp_c', 'max_temp_c', 'precipitation_mm','avg_wind_dir_deg', 'avg_wind_speed_kmh']] = imputer.fit_transform(historical_weather[['avg_temp_c', 'min_temp_c', 'max_temp_c', 'precipitation_mm','avg_wind_dir_deg', 'avg_wind_speed_kmh']])

In [9]:
historical_weather['temp_range'] = historical_weather['max_temp_c'] - historical_weather['min_temp_c']
historical_weather['cum_precip'] = historical_weather['precipitation_mm'].cumsum()
historical_weather['temp_precip_interaction'] = historical_weather['avg_temp_c'] * historical_weather['precipitation_mm']

# Temperature anomalies
historical_weather['temp_anomaly'] = historical_weather['avg_temp_c'] - historical_weather['avg_temp_c'].mean()

# Precipitation anomalies
historical_weather['precip_anomaly'] = historical_weather['precipitation_mm'] - historical_weather['precipitation_mm'].mean()

# Complex interactions
historical_weather['temp_precip_wind_interaction'] = historical_weather['avg_temp_c'] * historical_weather['precipitation_mm'] * historical_weather['avg_wind_speed_kmh']


In [10]:
historical_weather['rolling_mean_temp'] = historical_weather['avg_temp_c'].rolling(window=7).mean()
historical_weather['rolling_std_temp'] = historical_weather['avg_temp_c'].rolling(window=7).std()
historical_weather['rolling_sum_precip'] = historical_weather['precipitation_mm'].rolling(window=7).sum()

In [11]:
historical_weather['temp_deviation'] = historical_weather['avg_temp_c'] - historical_weather['avg_temp_c'].rolling(window=30).mean()
# Day-to-day temperature change
historical_weather['temp_day_change'] = historical_weather['avg_temp_c'] - historical_weather['avg_temp_c'].shift(1)
    
# Heavy precipitation indicator
historical_weather['heavy_precip'] = (historical_weather['precipitation_mm'] > 10).astype(int)
    
# Wind speed categories (example bins: calm, breeze, gale)
bins = [0, 5, 15, 30, np.inf]
labels = ['Calm', 'Breeze', 'Gale', 'Storm']
historical_weather['wind_speed_category'] = pd.cut(historical_weather['avg_wind_speed_kmh'], bins=bins, labels=labels)

# Wind direction sine and cosine components
historical_weather['wind_dir_sin'] = np.sin(np.radians(historical_weather['avg_wind_dir_deg']))
historical_weather['wind_dir_cos'] = np.cos(np.radians(historical_weather['avg_wind_dir_deg']))

# Temperature percentile rank
historical_weather['temp_percentile'] = historical_weather['avg_temp_c'].rank(pct=True)

# Cumulative moving average
historical_weather['cum_avg_temp'] = historical_weather['avg_temp_c'].expanding().mean()

In [12]:
day_of_year = historical_weather['date'].dt.dayofyear
historical_weather['fourier_sin'] = np.sin(2 * np.pi * day_of_year / 365.25)
historical_weather['fourier_cos'] = np.cos(2 * np.pi * day_of_year / 365.25)

In [13]:
def create_features(df):
    """
    Create time series features based on time series index.
    """
    df = df.copy()
    df['dayofweek'] = df['date'].dt.dayofweek
    df['quarter'] = df['date'].dt.quarter
    df['month'] = df['date'].dt.month
    df['year'] = df['date'].dt.year
    df['dayofyear'] = df['date'].dt.dayofyear
    df['dayofmonth'] = df['date'].dt.day
    df['weekofyear'] = df['date'].dt.isocalendar().week
    df['date_offset'] = (df.date.dt.month*100 + df.date.dt.day - 320)%1300

    df['season'] = pd.cut(df['date_offset'], [0, 300, 602, 900, 1300], 
                          labels=['Spring', 'Summer', 'Fall', 'Winter'])
    df['season'] = df['season'].cat.codes
    return df

historical_weather = create_features(historical_weather)

In [14]:
def create_lag_features(df, lags, cols):
    for col in cols:
        for lag in lags:
            df[f'{col}_lag_{lag}'] = df.groupby('city_id')[col].shift(lag)
    return df

In [15]:
lag_features = ['avg_temp_c']
lags = [1, 2, 3, 4, 5, 6, 7]  # 1-day, 2-day, 3-day, 7-day, 14-day, and 30-day lags
historical_weather = create_lag_features(historical_weather, lags, lag_features)

In [16]:
historical_weather

,city_id,date,avg_temp_c,min_temp_c,max_temp_c,precipitation_mm,avg_wind_dir_deg,avg_wind_speed_kmh,temp_range,cum_precip,...,weekofyear,date_offset,season,avg_temp_c_lag_1,avg_temp_c_lag_2,avg_temp_c_lag_3,avg_temp_c_lag_4,avg_temp_c_lag_5,avg_temp_c_lag_6,avg_temp_c_lag_7
0,1,2014-01-01,6.6,-1.4,11.6,3.679964,168.000000,6.2,13.0,3.679964,...,1,1081,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,2014-01-02,9.3,6.3,13.3,3.679964,155.000000,10.0,7.0,7.359929,...,1,1082,3,6.6,NaN,NaN,NaN,NaN,NaN,NaN
2,1,2014-01-03,7.6,1.9,14.0,3.679964,175.650289,5.8,12.1,11.039893,...,1,1083,3,9.3,6.6,NaN,NaN,NaN,NaN,NaN
3,1,2014-01-04,7.6,3.9,13.3,3.679964,291.000000,11.3,9.4,14.719857,...,1,1084,3,7.6,9.3,6.6,NaN,NaN,NaN,NaN
4,1,2014-01-05,8.6,0.5,16.9,3.679964,175.650289,5.0,16.4,18.399821,...,1,1085,3,7.6,7.6,9.3,6.6,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
182333,112,2018-12-27,22.0,15.9,27.9,0.000000,100.000000,13.2,12.0,670978.229896,...,52,907,3,22.3,21.9,20.6,19.3,18.5,19.2,21.0
182334,112,2018-12-28,21.9,14.9,28.2,0.000000,91.000000,12.6,13.3,670978.229896,...,52,908,3,22.0,22.3,21.9,20.6,19.3,18.5,19.2
182335,112,2018-12-29,22.4,16.3,28.2,0.000000,61.000000,14.2,11.9,670978.229896,...,52,909,3,21.9,22.0,22.3,21.9,20.6,19.3,18.5
182336,112,2018-12-30,21.6,18.5,26.6,1.600000,70.000000,17.0,8.1,670979.829896,...,52,910,3,22.4,21.9,22.0,22.3,21.9,20.6,19.3


In [17]:
historical_weather = historical_weather.dropna()

In [18]:
# scaler = StandardScaler()
# scaler = scaler.fit(historical_weather[['avg_temp_c', 'min_temp_c', 'max_temp_c', 'precipitation_mm','avg_wind_dir_deg', 'avg_wind_speed_kmh']])
# historical_weather_scaled = scaler.transform(historical_weather[['avg_temp_c', 'min_temp_c', 'max_temp_c', 'precipitation_mm','avg_wind_dir_deg', 'avg_wind_speed_kmh']])

In [19]:
# def prepare_data_for_lstm(data, n_past=14, n_future=7):
#     X, Y = [], []
#     for i in range(n_past, len(data) - n_future + 1):
#         X.append(data[i - n_past:i, :])  # past days data
#         Y.append(data[i:i + n_future, 0])  # future days target (avg_temp_c)
#     return np.array(X), np.array(Y)

# trainX, trainY = prepare_data_for_lstm(historical_weather_scaled)

In [20]:
# # Define the LSTM model
# model = Sequential()
# model.add(LSTM(64, activation='relu', input_shape=(trainX.shape[1], trainX.shape[2]), return_sequences=True))
# model.add(LSTM(32, activation='relu', return_sequences=False))
# model.add(Dropout(0.2))
# model.add(Dense(trainY.shape[1]))  # Output layer for predicting avg_temp_c

# model.compile(optimizer='adam', loss='mse')
# model.summary()

In [21]:
# history = model.fit(trainX, trainY, epochs=10, batch_size=16, validation_split=0.1, verbose=1)

In [22]:
# plt.plot(history.history['loss'], label='Training loss')
# plt.plot(history.history['val_loss'], label='Validation loss')
# plt.legend()

In [23]:
# predictions = []
# for index, row in test_data.iterrows():
#     city_id = row['city_id']
#     date = row['date']
#     city_data = historical_weather[(historical_weather['city_id'] == city_id) & (historical_weather['date'] <= date)]
#     city_data_scaled = scaler.transform(city_data[['avg_temp_c', 'min_temp_c', 'max_temp_c', 'precipitation_mm','avg_wind_dir_deg', 'avg_wind_speed_kmh']])
    
#     # Reshape data for LSTM input
#     X = np.expand_dims(city_data_scaled[-14:, :], axis=0)  # Using last 14 days data
#     prediction = model.predict(X)
#     predictions.append(prediction[0][0])  # Predicted avg_temp_c for the date

# # Create submission dataframe
# submission = pd.DataFrame({
#     'submission_ID': test_data['submission_ID'],
#     'avg_temp_c': predictions
# })

# # Save submission to CSV file
# submission.to_csv('my_submission.csv', index=False)

In [24]:
# Load and preprocess the data
# df = pd.read_csv('/mnt/data/historical_weather.csv')
# df['date'] = pd.to_datetime(df['date'])
# df = create_features(df)

from sklearn.preprocessing import MinMaxScaler

# Function to prepare data for LSTM
def prepare_lstm_data(df, city_id, sequence_length=30):
    city_df = df[df['city_id'] == city_id].sort_values('date')
    features = ['avg_temp_c', 'min_temp_c', 'max_temp_c', 'precipitation_mm','avg_wind_dir_deg', 'avg_wind_speed_kmh','dayofweek', 'quarter', 'month','year', 'dayofyear','dayofmonth','weekofyear','avg_temp_c_lag_1','avg_temp_c_lag_2',
                'avg_temp_c_lag_3','avg_temp_c_lag_4','avg_temp_c_lag_5','avg_temp_c_lag_6','avg_temp_c_lag_7']
    city_df = city_df[features].dropna()
    
    scaler = MinMaxScaler()
    city_scaled = scaler.fit_transform(city_df)
    
    X, y = [], []
    for i in range(sequence_length, len(city_scaled) - 7):
        X.append(city_scaled[i-sequence_length:i])
        y.append(city_scaled[i:i+7, 0])  # Predict next 7 days of avg_temp_c

    X, y = np.array(X), np.array(y)
    return X, y, scaler

# Function to build and train LSTM model
def train_lstm(X_train, y_train):
    model = Sequential()
    model.add(LSTM(units=64, activation='relu', return_sequences=True, input_shape=(X_train.shape[1], X_train.shape[2])))
    model.add(LSTM(32, activation='relu', return_sequences=False))
    model.add(Dropout(0.2))
    # model.add(LSTM(units=50))
    model.add(Dense(7))  # Predicting 7 days
    model.compile(loss='mean_squared_error', optimizer='adam')
    
    model.fit(X_train, y_train, epochs=20, batch_size=32, validation_split=0.1)
    return model

# Function to make predictions
def predict_next_7_days(model, X, scaler, num_features):
    prediction_scaled = model.predict(X[-1].reshape(1, X.shape[1], X.shape[2]))
    padding = np.zeros((7, num_features - 1))
    prediction = scaler.inverse_transform(np.hstack([prediction_scaled.reshape(-1, 1), padding]))[:, 0]
    return prediction

# List to store predictions for all cities
all_cities_predictions = []

# Iterate through all unique city IDs
unique_city_ids = historical_weather['city_id'].unique()

for city_id in unique_city_ids:
    X, y, scaler = prepare_lstm_data(historical_weather, city_id)
    if X.shape[0] > 0:  # Ensure there is enough data for training
        model = train_lstm(X, y)
        num_features = X.shape[2]
        predictions = predict_next_7_days(model, X, scaler, num_features)
        all_cities_predictions.append({
            'city_id': city_id,
            'predictions': predictions
        })

# Display the predictions
all_cities_predictions

Epoch 1/20
50/50 [==============================] - 3s 18ms/step - loss: 0.1128 - val_loss: 0.0105
Epoch 2/20
50/50 [==============================] - 1s 11ms/step - loss: 0.0386 - val_loss: 0.0116
Epoch 3/20
50/50 [==============================] - 1s 11ms/step - loss: 0.0311 - val_loss: 0.0082
Epoch 4/20
50/50 [==============================] - 1s 12ms/step - loss: 0.0268 - val_loss: 0.0076
Epoch 5/20
50/50 [==============================] - 1s 11ms/step - loss: 0.0230 - val_loss: 0.0078
Epoch 6/20
50/50 [==============================] - 1s 11ms/step - loss: 0.0207 - val_loss: 0.0069
Epoch 7/20
50/50 [==============================] - 1s 11ms/step - loss: 0.0193 - val_loss: 0.0069
Epoch 8/20
50/50 [==============================] - 1s 11ms/step - loss: 0.0186 - val_loss: 0.0081
Epoch 9/20
50/50 [==============================] - 1s 11ms/step - loss: 0.0172 - val_loss: 0.0069
Epoch 10/20
50/50 [==============================] - 1s 13ms/step - loss: 0.0159 - val_loss: 0.0104
Epoch 11/

[{'city_id': 1,
  'predictions': array([10.24110413, 10.39927899, 10.72591119, 10.37826245, 10.40325254,
         10.93790186, 10.53496541])},
 {'city_id': 2,
  'predictions': array([14.58005676, 14.72816194, 14.41592942, 14.65113652, 14.63502862,
         14.2526975 , 14.00981303])},
 {'city_id': 3,
  'predictions': array([26.02868676, 26.09012638, 26.07149909, 26.01082401, 25.94739596,
         26.05934938, 26.02102321])},
 {'city_id': 4,
  'predictions': array([-1.71822585, -1.58493906, -1.7764756 , -1.91777437, -1.39179989,
         -1.96797415, -1.53149269])},
 {'city_id': 5,
  'predictions': array([24.96926209, 24.74784573, 24.68769164, 24.60110536, 24.96847332,
         25.0446254 , 25.01140311])},
 {'city_id': 7,
  'predictions': array([15.46128756, 15.0615981 , 14.76375857, 15.00548873, 15.06998532,
         14.8848446 , 15.37396638])},
 {'city_id': 8,
  'predictions': array([7.40900193, 8.3610588 , 7.88223061, 9.22397327, 7.94546878,
         6.90495113, 9.21510401])},
 {'cit

In [25]:
len(all_cities_predictions)

100

In [26]:
results = []

for city_prediction in all_cities_predictions:
    city_id = city_prediction['city_id']
    predictions = city_prediction['predictions']
    for i, pred in enumerate(predictions):
        results.append({'city_id': city_id, 'day': i + 1, 'predicted_avg_temp_c': pred})

df_predictions = pd.DataFrame(results)

In [27]:
df_predictions

,city_id,day,predicted_avg_temp_c
0,1,1,10.241104
1,1,2,10.399279
2,1,3,10.725911
3,1,4,10.378262
4,1,5,10.403253
...,...,...,...
695,112,3,19.439725
696,112,4,19.440849
697,112,5,19.655160
698,112,6,19.639201


In [28]:
submission = pd.DataFrame({
    'submission_ID': test_data['submission_ID'],
    'avg_temp_c': df_predictions['predicted_avg_temp_c']
})

# Save submission to CSV file
submission.to_csv('submission_LSTM_6.csv', index=False)